A template of the RL training

In [1]:
import numpy as np
import pandas as pd

from pathlib import Path
from datetime import datetime

from utils import print_log

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
# load the created dataset
dataset_folder_path = Path("dataset", "20250707", "split")

In [4]:
# copied from 03_data_split.ipynb

# Helper functions for the new split folder structure
def load_split_data_from_folder(split_folder, split_type='train'):
    """Load aggregate data from split folder"""
    segments = []
    with open(split_folder / f'{split_type}_segments.txt', 'r') as f:
        for line in f:
            start_str, end_str = line.strip().split(' - ')
            start = datetime.fromisoformat(start_str)
            end = datetime.fromisoformat(end_str)
            segments.append((start, end))
    
    df = pd.read_pickle(split_folder / f'{split_type}_aggregate_df.pkl')
    return segments, df

def load_signatures_from_split_folder(split_folder, split_type, appliance):
    """Load appliance signatures from split folder"""
    sig_path = split_folder / 'load_signature_library' / split_type / appliance / 'load_signatures.pkl'
    ranges_path = split_folder / 'load_signature_library' / split_type / appliance / 'selected_ranges.txt'
    
    if not sig_path.exists():
        return pd.DataFrame(), []
    
    signatures_df = pd.read_pickle(sig_path)
    ranges = []
    if ranges_path.exists():
        with open(ranges_path, 'r') as f:
            for line in f:
                start, end = map(int, line.strip().split(','))
                ranges.append((start, end))
    
    return signatures_df, ranges

In [5]:
aggregate_load_segments_train, aggregate_load_df_train = load_split_data_from_folder(dataset_folder_path, 'train')
aggregate_load_segments_test, aggregate_load_df_test = load_split_data_from_folder(dataset_folder_path, 'test') 
aggregate_load_segments_validation, aggregate_load_df_validation = load_split_data_from_folder(dataset_folder_path, 'val')

In [6]:
aggregate_load_df_train

,timestamp,aggregate,datetime,washing_machine,dishwasher,fridge,kettle,microwave,toaster,tv,htpc,gas_oven,kitchen_lights
0,1.357603e+09,234.0,2013-01-08 00:00:05+00:00,0.0,1.0,0.0,1.0,1.0,0.0,1.0,69.0,NaN,0.0
1,1.357603e+09,231.0,2013-01-08 00:00:11+00:00,0.0,1.0,0.0,1.0,1.0,0.0,1.0,70.0,NaN,0.0
2,1.357603e+09,234.0,2013-01-08 00:00:17+00:00,0.0,1.0,0.0,1.0,1.0,0.0,1.0,70.0,NaN,0.0
3,1.357603e+09,232.0,2013-01-08 00:00:23+00:00,0.0,1.0,0.0,1.0,1.0,0.0,1.0,68.0,NaN,0.0
4,1.357603e+09,232.0,2013-01-08 00:00:30+00:00,0.0,1.0,0.0,1.0,1.0,0.0,1.0,70.0,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2231636,1.388448e+09,178.0,2013-12-30 23:59:35+00:00,0.0,1.0,0.0,1.0,1.0,0.0,1.0,2.0,3.0,0.0
2231637,1.388448e+09,177.0,2013-12-30 23:59:41+00:00,0.0,1.0,0.0,1.0,1.0,0.0,1.0,2.0,3.0,0.0
2231638,1.388448e+09,178.0,2013-12-30 23:59:47+00:00,0.0,1.0,0.0,1.0,1.0,0.0,1.0,2.0,3.0,0.0
2231639,1.388448e+09,178.0,2013-12-30 23:59:53+00:00,0.0,1.0,0.0,1.0,1.0,0.0,1.0,2.0,3.0,0.0


In [7]:
aggregate_load_segments_train

[(datetime.datetime(2013, 1, 8, 0, 0, tzinfo=datetime.timezone.utc),
  datetime.datetime(2013, 1, 10, 23, 59, 59, 999999, tzinfo=datetime.timezone.utc)),
 (datetime.datetime(2013, 2, 27, 0, 0, tzinfo=datetime.timezone.utc),
  datetime.datetime(2013, 2, 28, 23, 59, 59, 999999, tzinfo=datetime.timezone.utc)),
 (datetime.datetime(2013, 3, 8, 0, 0, tzinfo=datetime.timezone.utc),
  datetime.datetime(2013, 3, 10, 23, 59, 59, 999999, tzinfo=datetime.timezone.utc)),
 (datetime.datetime(2013, 3, 28, 0, 0, tzinfo=datetime.timezone.utc),
  datetime.datetime(2013, 3, 31, 23, 59, 59, 999999, tzinfo=datetime.timezone.utc)),
 (datetime.datetime(2013, 3, 22, 0, 0, tzinfo=datetime.timezone.utc),
  datetime.datetime(2013, 3, 26, 23, 59, 59, 999999, tzinfo=datetime.timezone.utc)),
 (datetime.datetime(2013, 4, 8, 0, 0, tzinfo=datetime.timezone.utc),
  datetime.datetime(2013, 4, 10, 23, 59, 59, 999999, tzinfo=datetime.timezone.utc)),
 (datetime.datetime(2013, 4, 26, 0, 0, tzinfo=datetime.timezone.utc),
  d

In [8]:
from rl_env.env_data_loader import SmartMeterDataLoader

sm_dl = SmartMeterDataLoader(
    aggregate_load_segments=aggregate_load_segments_train,
    aggregate_load_df=aggregate_load_df_train
)

sm_dl.get_divided_segments_length()

162

In [9]:
sm_dl.divided_segments[7]

array([datetime.datetime(2013, 3, 10, 0, 0, tzinfo=datetime.timezone.utc),
       datetime.datetime(2013, 3, 10, 23, 59, 59, 999999, tzinfo=datetime.timezone.utc)],
      dtype=object)

In [10]:
load_segment = sm_dl.get_aggregate_load_segment(13)

load_segment

,timestamp,aggregate,datetime
104747,1.363997e+09,335.0,2013-03-23 00:00:05+00:00
104748,1.363997e+09,336.0,2013-03-23 00:00:11+00:00
104749,1.363997e+09,333.0,2013-03-23 00:00:17+00:00
104750,1.363997e+09,334.0,2013-03-23 00:00:24+00:00
104751,1.363997e+09,331.0,2013-03-23 00:00:30+00:00
...,...,...,...
118501,1.364083e+09,179.0,2013-03-23 23:59:30+00:00
118502,1.364083e+09,171.0,2013-03-23 23:59:37+00:00
118503,1.364083e+09,171.0,2013-03-23 23:59:43+00:00
118504,1.364083e+09,171.0,2013-03-23 23:59:49+00:00


In [11]:
load_segment['datetime'] = pd.to_datetime(load_segment['timestamp'], unit='s')

load_segment

,timestamp,aggregate,datetime
104747,1.363997e+09,335.0,2013-03-23 00:00:05
104748,1.363997e+09,336.0,2013-03-23 00:00:11
104749,1.363997e+09,333.0,2013-03-23 00:00:17
104750,1.363997e+09,334.0,2013-03-23 00:00:24
104751,1.363997e+09,331.0,2013-03-23 00:00:30
...,...,...,...
118501,1.364083e+09,179.0,2013-03-23 23:59:30
118502,1.364083e+09,171.0,2013-03-23 23:59:37
118503,1.364083e+09,171.0,2013-03-23 23:59:43
118504,1.364083e+09,171.0,2013-03-23 23:59:49


(Optional) Load the pre-trained H-network and related components

In final product, the H-network should be trained along with the DDQL/PPO agent

In [12]:
import torch
from model.H_network.h_network import HNetwork

h_network_datetime = datetime(2025, 7, 12)

h_network_path = Path("model_trained", f"h_network_{h_network_datetime.strftime('%Y%m%d')}.pth")

h_network = HNetwork(2, 44, 1)
h_network.load_state_dict(torch.load(h_network_path))
h_network.eval()

HNetwork(
  (LSTM_1): LSTM(2, 44, batch_first=True, bidirectional=True)
  (ac1): Tanh()
  (LSTM_2): LSTM(88, 1, batch_first=True, bidirectional=True)
  (ac2): Tanh()
  (fc): Linear(in_features=2, out_features=1, bias=True)
)

In [13]:
h_network_stdscaler_path = Path("model_trained", f"h_network_standardscaler_{h_network_datetime.strftime('%Y%m%d')}.pkl")
import joblib
h_network_stdscaler = joblib.load(h_network_stdscaler_path)

Create the environment

In [14]:
import sys
sys.path.append(str(Path('rl_env')))

from rl_env.hrl_env import SmartMeterWorld

env = SmartMeterWorld(
    aggregate_load_df=load_segment,
    render_mode="human",
)

env.set_h_network(h_network)
env.set_h_network_stdscaler(h_network_stdscaler)

[2025-07-12 02:01:26:508] [SmartMeterWorld] Render mode set to 'human'. Render server at 127.0.0.1:50007. render_connected: True. render_client_socket: <socket.socket fd=69, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0, laddr=('127.0.0.1', 35610), raddr=('127.0.0.1', 50007)>


In [15]:
from gymnasium.utils.env_checker import check_env

# This will catch many common issues
try:
    check_env(env)
    print("Environment passes all checks!")
except Exception as e:
    print(f"Environment has issues: {e}")

Environment passes all checks!


/root/miniforge3/envs/smartmeter/lib/python3.9/site-packages/gymnasium/utils/passive_env_checker.py:334: UserWarning: WARN: No render fps was declared in the environment (env.metadata['render_fps'] is None or not defined), rendering may occur at inconsistent fps.
  logger.warn(
/root/miniforge3/envs/smartmeter/lib/python3.9/site-packages/gymnasium/utils/env_checker.py:434: UserWarning: WARN: Not able to test alternative render modes due to the environment not having a spec. Try instantiating the environment through `gymnasium.make`
  logger.warn(


In [16]:
obs = env.reset()
obs

({'aggregate_load': array([335.], dtype=float32),
  'battery_soc': array([0.05382102], dtype=float32),
  'timestamp_features': array([-0.5       ,  0.33333334, -0.25      ], dtype=float32)},
 {})

In [17]:
env.reset_render_window()

In [18]:
# initialize a PPO agent
from stable_baselines3 import PPO

rl_datetime = datetime.now()
tensorboard_log_path = Path("rl_model", "PPO", f"{rl_datetime.strftime('%Y%m%d_%H%M%S')}")

rl_model = PPO(
    "MultiInputPolicy", 
    env, 
    verbose=2,
    tensorboard_log=tensorboard_log_path
)

rl_model.learn(
    total_timesteps=300000,
    progress_bar=True,
    tb_log_name="PPO_SmartMeterWorld"
)

2025-07-12 02:01:26.833878: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-12 02:01:26.841661: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752282086.850399  309627 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752282086.852950  309627 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1752282086.859580  309627 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Logging to rl_model/PPO/20250712_020127/PPO_SmartMeterWorld_1


Output()

-----------------------------
| time/              |      |
|    fps             | 101  |
|    iterations      | 1    |
|    time_elapsed    | 20   |
|    total_timesteps | 2048 |
-----------------------------


-----------------------------------------
| time/                   |             |
|    fps                  | 85          |
|    iterations           | 2           |
|    time_elapsed         | 47          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.002185715 |
|    clip_fraction        | 0.000586    |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.42       |
|    explained_variance   | 2.98e-06    |
|    learning_rate        | 0.0003      |
|    loss                 | 9.66e+05    |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.000123   |
|    std                  | 0.992       |
|    value_loss           | 1.99e+06    |
-----------------------------------------


...

------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 1.38e+04     |
|    ep_rew_mean          | 7.48e+06     |
| time/                   |              |
|    fps                  | 58           |
|    iterations           | 143          |
|    time_elapsed         | 5016         |
|    total_timesteps      | 292864       |
| train/                  |              |
|    approx_kl            | 0.0040676566 |
|    clip_fraction        | 0.0145       |
|    clip_range           | 0.2          |
|    entropy_loss         | -1.39        |
|    explained_variance   | 1.19e-07     |
|    learning_rate        | 0.0003       |
|    loss                 | 3.14e+06     |
|    n_updates            | 1420         |
|    policy_gradient_loss | -0.000784    |
|    std                  | 0.965        |
|    value_loss           | 2.14e+07     |
------------------------------------------


-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 1.38e+04    |
|    ep_rew_mean          | 7.48e+06    |
| time/                   |             |
|    fps                  | 58          |
|    iterations           | 144         |
|    time_elapsed         | 5046        |
|    total_timesteps      | 294912      |
| train/                  |             |
|    approx_kl            | 0.002895398 |
|    clip_fraction        | 0.00317     |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.38       |
|    explained_variance   | 0           |
|    learning_rate        | 0.0003      |
|    loss                 | 4.32e+06    |
|    n_updates            | 1430        |
|    policy_gradient_loss | -0.000555   |
|    std                  | 0.967       |
|    value_loss           | 8.8e+06     |
-----------------------------------------


------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 1.38e+04     |
|    ep_rew_mean          | 7.48e+06     |
| time/                   |              |
|    fps                  | 58           |
|    iterations           | 145          |
|    time_elapsed         | 5082         |
|    total_timesteps      | 296960       |
| train/                  |              |
|    approx_kl            | 0.0007584458 |
|    clip_fraction        | 0            |
|    clip_range           | 0.2          |
|    entropy_loss         | -1.38        |
|    explained_variance   | 0            |
|    learning_rate        | 0.0003       |
|    loss                 | 1.37e+07     |
|    n_updates            | 1440         |
|    policy_gradient_loss | -0.000238    |
|    std                  | 0.964        |
|    value_loss           | 2.69e+07     |
------------------------------------------


-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 1.38e+04    |
|    ep_rew_mean          | 7.48e+06    |
| time/                   |             |
|    fps                  | 58          |
|    iterations           | 146         |
|    time_elapsed         | 5122        |
|    total_timesteps      | 299008      |
| train/                  |             |
|    approx_kl            | 0.003507853 |
|    clip_fraction        | 0.00503     |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.38       |
|    explained_variance   | 0           |
|    learning_rate        | 0.0003      |
|    loss                 | 2.78e+07    |
|    n_updates            | 1450        |
|    policy_gradient_loss | -0.00179    |
|    std                  | 0.964       |
|    value_loss           | 5.51e+07    |
-----------------------------------------


------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 1.38e+04     |
|    ep_rew_mean          | 7.48e+06     |
| time/                   |              |
|    fps                  | 58           |
|    iterations           | 147          |
|    time_elapsed         | 5167         |
|    total_timesteps      | 301056       |
| train/                  |              |
|    approx_kl            | 9.829071e-05 |
|    clip_fraction        | 0            |
|    clip_range           | 0.2          |
|    entropy_loss         | -1.38        |
|    explained_variance   | 0            |
|    learning_rate        | 0.0003       |
|    loss                 | 4.61e+07     |
|    n_updates            | 1460         |
|    policy_gradient_loss | -4.25e-05    |
|    std                  | 0.964        |
|    value_loss           | 9.19e+07     |
------------------------------------------


In [19]:
obs, info = env.reset()
for i in range(1):
    done = False
    while not done:
        action, _states = rl_model.predict(obs, deterministic=True)
        obs, reward, done, truncated, info = env.step(action)
        print_log(f"Step: {env.current_step}, Action: {action}, Reward: {reward}")
        env.render()

[2025-07-12 04:40:12:693] Step: 1, Action: [0.6829988], Reward: 0.28550860819745066
[2025-07-12 04:40:12:706] Step: 2, Action: [0.6830041], Reward: 0.029417896038015683
[2025-07-12 04:40:12:715] Step: 3, Action: [0.6829878], Reward: 0.0020240538809208407
[2025-07-12 04:40:12:721] Step: 4, Action: [0.6829932], Reward: 0.00039408203559865556
[2025-07-12 04:40:12:726] Step: 5, Action: [0.6829765], Reward: 0.00039342130331570906
[2025-07-12 04:40:12:733] Step: 6, Action: [0.6829765], Reward: 0.0029685337984214225
[2025-07-12 04:40:12:738] Step: 7, Action: [0.68294775], Reward: 0.016577786414345107
[2025-07-12 04:40:12:743] Step: 8, Action: [0.68293583], Reward: 0.04486150693547726
[2025-07-12 04:40:12:748] Step: 9, Action: [0.68292356], Reward: 0.07381703701988857
[2025-07-12 04:40:12:753] Step: 10, Action: [0.6828985], Reward: 0.0961358482132223
[2025-07-12 04:40:12:761] Step: 11, Action: [0.6829048], Reward: 0.11127642069892088
[2025-07-12 04:40:12:767] Step: 12, Action: [0.68287903], Re

In [20]:
env.aggregate_load_df

,timestamp,aggregate,datetime,grid_load,battery_soc
104747,1.363997e+09,335.0,2013-03-23 00:00:05,3066.995106,0.184338
104748,1.363997e+09,336.0,2013-03-23 00:00:11,3068.016325,0.184907
104749,1.363997e+09,333.0,2013-03-23 00:00:17,3064.951237,0.185476
104750,1.363997e+09,334.0,2013-03-23 00:00:24,3065.972694,0.18614
104751,1.363997e+09,331.0,2013-03-23 00:00:30,3062.905937,0.18671
...,...,...,...,...,...
118501,1.364083e+09,179.0,2013-03-23 23:59:30,2906.402687,1.0
118502,1.364083e+09,171.0,2013-03-23 23:59:37,2897.657391,1.0
118503,1.364083e+09,171.0,2013-03-23 23:59:43,2897.657391,1.0
118504,1.364083e+09,171.0,2013-03-23 23:59:49,2897.657391,1.0


In [21]:
# save the graph

env.save_graph(
    str(Path("rl_model", "PPO", f"{rl_datetime.strftime('%Y%m%d_%H%M%S')}", "graph.png"))
)

In [22]:
env.close()

[2025-07-12 04:44:05:356] [SmartMeterWorld] Environment closed.


In [23]:
# save the model
rl_model_path = Path("rl_model", "PPO", f"{rl_datetime.strftime('%Y%m%d_%H%M%S')}", "rl_model.zip")
rl_model.save(rl_model_path)

---

In [ ]:
# load the model & environment
import sys
sys.path.append(str(Path('rl_env')))

from rl_env.hrl_env import SmartMeterWorld
from stable_baselines3 import PPO

env = SmartMeterWorld(
    aggregate_load_df=load_segment,
    render_mode="human",
)

env.set_h_network(h_network)
env.set_h_network_stdscaler(h_network_stdscaler)

rl_model_path = Path("rl_model", "PPO", f"{datetime(2025,7,10,19,18,57).strftime('%Y%m%d_%H%M%S')}", "rl_model.zip")
rl_model_loaded = PPO.load(rl_model_path, env=env)

[2025-07-11 17:41:15:349] [SmartMeterWorld] Render mode set to 'human'. Render server at 127.0.0.1:50007. render_connected: True. render_client_socket: <socket.socket fd=85, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0, laddr=('127.0.0.1', 57334), raddr=('127.0.0.1', 50007)>
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [ ]:
obs, info = env.reset()
for i in range(1):
    done = False
    while not done:
        action, _states = rl_model_loaded.predict(obs, deterministic=True)
        obs, reward, done, truncated, info = env.step(action)
        print_log(f"Step: {env.current_step}, Action: {action}, Reward: {reward}")
        env.render()

[2025-07-11 17:43:49:737] Step: 1, Action: [0.8852709], Reward: 0.11131098730440935
[2025-07-11 17:45:38:966] Step: 2, Action: [0.8852702], Reward: 0.5481597745709021
[2025-07-11 17:45:47:193] Step: 3, Action: [0.88527256], Reward: 0.7274080372309486
[2025-07-11 17:45:47:202] Step: 4, Action: [0.88527185], Reward: 0.9513516866327525
[2025-07-11 17:45:47:211] Step: 5, Action: [0.885274], Reward: 1.2329133591758012
[2025-07-11 17:45:47:217] Step: 6, Action: [0.88527405], Reward: 1.4986055812901657
[2025-07-11 17:45:47:222] Step: 7, Action: [0.8852769], Reward: 1.7794962723778884
[2025-07-11 17:45:47:234] Step: 8, Action: [0.8852778], Reward: 2.0722826678674617
[2025-07-11 17:45:47:243] Step: 9, Action: [0.88527864], Reward: 2.3641625840023757
[2025-07-11 17:45:47:253] Step: 10, Action: [0.8852796], Reward: 2.6564700872594185
[2025-07-11 17:45:47:268] Step: 11, Action: [0.8852795], Reward: 2.941596050857504
[2025-07-11 17:45:47:277] Step: 12, Action: [0.88527995], Reward: 3.22153648909596

In [ ]:
env.save_graph(
    str(Path("rl_model", "PPO", f"{rl_datetime.strftime('%Y%m%d_%H%M%S')}", "graph2.png"))
)